In [34]:
import os
from typing import Optional

from dotenv import load_dotenv
from langchain_core.utils.utils import secret_from_env
from langchain_openai import ChatOpenAI
from pydantic import Field, SecretStr

load_dotenv()

True

In [42]:
class ChatOpenRouter(ChatOpenAI):
    openai_api_key: Optional[SecretStr] = Field(
        alias="api_key",
        default_factory=secret_from_env("OPENROUTER_API_KEY", default=None),
    )

    @property
    def lc_secrets(self) -> dict[str, str]:
        return {"openai_api_key": "OPENROUTER_API_KEY"}

    def __init__(self, openai_api_key: Optional[str] = None, **kwargs):
        openai_api_key = openai_api_key or os.environ.get("OPENROUTER_API_KEY")
        super().__init__(base_url="https://openrouter.ai/api/v1",
                         openai_api_key=openai_api_key,
                         **kwargs)

In [43]:
openrouter_model = ChatOpenRouter(
    model_name="openrouter/sonoma-dusk-alpha"
)

In [38]:
openrouter_model.invoke("Hello, world!").content

"Hello! I'm Sonoma, built by Oak AI. What's on your mind today?"

In [44]:
res = openrouter_model.invoke("Can you speak Chinese?")

In [45]:
type(res)

langchain_core.messages.ai.AIMessage

In [46]:
res.content

'是的，我可以说中文！我是Sonoma，由Oak AI构建的AI助手，能用中文流利交流。你有什么问题或想聊什么吗？'

In [47]:
res

AIMessage(content='是的，我可以说中文！我是Sonoma，由Oak AI构建的AI助手，能用中文流利交流。你有什么问题或想聊什么吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 34, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'openrouter/sonoma-dusk-alpha', 'system_fingerprint': None, 'id': 'gen-1757597371-Fhy6beyYfoBtbRDOaXwL', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--8bd621fb-f88d-4681-aa9e-eac011365a92-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [48]:
from langchain_core.output_parsers import StrOutputParser

In [49]:
chain = openrouter_model | StrOutputParser()
chain.invoke("Can you speak Chinese?")

'Yes, I can speak Chinese. For example: 你好！我是Sonoma，由Oak AI构建的AI助手。如果你有问题，用中文问我也没问题。'

In [51]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="name", description="用户的姓名"),
    ResponseSchema(name="age", description="用户的年龄")
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // 用户的姓名
	"age": string  // 用户的年龄
}
```


In [52]:
from string import printable


prompt = PromptTemplate(
    template="回答用户的问题，并返回Json格式：{question}\n{format_instructions}",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
chain = prompt | openrouter_model | parser
res = chain.invoke({"question": "我的名字叫张三，我今年20岁。"})

print(res)

{'name': '张三', 'age': '20'}
